In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,65547
2,Huelva,Confirmados PDIA 14 días,8190
3,Huelva,Tasa PDIA 14 días,"1595,9623516573456"
4,Huelva,Confirmados PDIA 7 días,3972
5,Huelva,Tasa PDIA 7 dias,"774,0125104741119"
6,Huelva,Total Confirmados,65764
7,Huelva,Curados,51519
8,Huelva,Fallecidos,431


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  65547.0


/tmp/ipykernel_103194/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  19205.0


/tmp/ipykernel_103194/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_103194/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_103194/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_103194/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 105 personas en los últimos 7 días 

Un positivo PCR cada 51 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,65547.0,8190.0,3972.0,513170.0,774.012510,1595.962352,1936.0
Huelva-Costa,38516.0,4772.0,2295.0,289548.0,792.614696,1648.085982,1128.0
Huelva (capital),19205.0,2844.0,1364.0,143837.0,948.295640,1977.238124,690.0
Condado-Campiña,19171.0,1893.0,900.0,156231.0,576.070050,1211.667339,517.0
Sierra de Huelva-Andévalo Central,7398.0,1509.0,770.0,67391.0,1142.585805,2239.171403,288.0
Aljaraque,2591.0,391.0,197.0,21474.0,917.388470,1820.806557,80.0
Palma del Condado (La),1795.0,149.0,92.0,10801.0,851.772984,1379.501898,71.0
Isla Cristina,3723.0,260.0,125.0,21393.0,584.303277,1215.350816,68.0
Moguer,2671.0,263.0,128.0,21867.0,585.356931,1202.725568,66.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,24.0,22.0,3.0,330.0,909.090909,6666.666667,0.0
Rosal de la Frontera,179.0,74.0,20.0,1697.0,1178.550383,4360.636417,12.0
Cerro de Andévalo (El),334.0,89.0,55.0,2327.0,2363.558229,3824.666953,26.0
Villanueva de las Cruces,46.0,14.0,5.0,387.0,1291.989664,3617.571059,3.0
Fuenteheridos,67.0,22.0,16.0,709.0,2256.699577,3102.961918,8.0
Campillo (El),197.0,60.0,34.0,2024.0,1679.841897,2964.426877,13.0
Jabugo,201.0,66.0,32.0,2260.0,1415.929204,2920.353982,14.0
Zalamea la Real,264.0,86.0,48.0,3054.0,1571.709234,2815.979044,18.0
Niebla,559.0,115.0,63.0,4116.0,1530.612245,2793.974733,40.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Hinojales,24.0,22.0,3.0,330.0,909.090909,6666.666667,0.0,0.136364
Paterna del Campo,388.0,50.0,10.0,3457.0,289.268152,1446.340758,6.0,0.200000
Rosal de la Frontera,179.0,74.0,20.0,1697.0,1178.550383,4360.636417,12.0,0.270270
Alosno,349.0,31.0,10.0,3933.0,254.258835,788.202390,4.0,0.322581
Berrocal,24.0,3.0,1.0,303.0,330.033003,990.099010,0.0,0.333333
Puerto Moral,29.0,3.0,1.0,278.0,359.712230,1079.136691,0.0,0.333333
Villanueva de las Cruces,46.0,14.0,5.0,387.0,1291.989664,3617.571059,3.0,0.357143
Santa Bárbara de Casa,64.0,11.0,4.0,1043.0,383.509108,1054.650048,3.0,0.363636
Villanueva de los Castillejos,298.0,35.0,13.0,2825.0,460.176991,1238.938053,6.0,0.371429
